<a href="https://colab.research.google.com/github/martinsotir/misc/blob/master/notebooks/2018-12-07-advent_of_code_18_day7_with_OR-Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Day 7 - Task scheduling with OR-Tools

In [0]:
!pip install -U ortools==6.10.6025

In [0]:
from ortools.sat.python import cp_model
import numpy as np
import re
import collections
import pandas as pd

In [0]:
def day_7_part1(instructions):

  # Convert instructions to (first, second) integer pairs:
  instr_re = re.compile('Step (\w+) must be finished before step (\w+) can begin\.')
  order_constraints = [list(map(ord, instr_re.match(line).groups())) for line in instructions.strip().splitlines()]
  
  # Job list infered from instructions
  jobs = {x for pair in order_constraints for x in pair}
  
  model = cp_model.CpModel()
  job_positions = {job: model.NewIntVar(0, len(jobs), f"pos_job#{chr(job)}") for job in sorted(jobs)}  
  
  # Important: try to enforce a greedy task priority using decision strategy (not sure if it's right):
  model.AddDecisionStrategy([job_positions[job] for job in sorted(jobs)], cp_model.CHOOSE_LOWEST_MIN, cp_model.SELECT_MIN_VALUE)
  
  # No overlap:
  model.AddAllDifferent(list(job_positions.values()))
  
  # Hard constraints for precedences:
  for (first, second) in order_constraints:
    model.Add(job_positions[first] < job_positions[second])
    
  solver = cp_model.CpSolver()
  
  solver.parameters.search_branching = cp_model.FIXED_SEARCH  # Required to enforce decision strategy

  status = solver.Solve(model)
  print('Status: ' + solver.StatusName(status))
  
  return ''.join(map(chr, sorted(job_positions, key=lambda x: solver.Value(job_positions[x]))))

In [0]:
instructions = """
Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin.
"""

In [0]:
day_7_part1(instructions)
#day_7_part1(open('input.txt').read())

Status: FEASIBLE


'CABDFE'

In [0]:
def day_7_part2(instructions, nb_elfs, min_task_duration):

  # Convert instructions to (first, second) integer pairs:
  instr_re = re.compile('Step (\w+) must be finished before step (\w+) can begin\.')
  order_constraints = [list(map(ord, instr_re.match(line).groups())) for line in instructions.strip().splitlines()]

  # Job list infered from instructions
  jobs = list(sorted({x for pair in order_constraints for x in pair}))

  # Compute job duration from position in alphabet
  jobs_duration = {job: min_task_duration + job - ord('A') + 1 for job in jobs}
  
  model = cp_model.CpModel()
  
  # Boolean task assigment matrix (elf x task)
  assigments = {(e, j) : model.NewBoolVar(f"E{e}_task#{chr(j)}") for e in range(nb_elfs) for j in jobs}
  
  # Each task assigned to one and only one elf:
  for j in jobs:
    model.AddSumConstraint([assigments[e, j] for e in range(nb_elfs)], 1, 1)
    
  # Worst case makespan:
  horizon = sum(jobs_duration.values())
  min_duration = min(jobs_duration.values())
  
  # For each elf, for each task, create optional interval variables:
  Task = collections.namedtuple('Task', 'start, end, interval')
  
  all_tasks = {}
  for e in range(nb_elfs):
    for job in jobs:
      start = model.NewIntVar(0, horizon - min_duration + 1, f'E{e}_start_{chr(job)}')
      end = model.NewIntVar(min_duration, horizon, f'E{e}_end_{chr(job)}')
      # model.Add(end == (start + jobs_duration[job])) ##not required, even slower when added!
      interval = model.NewOptionalIntervalVar(start, jobs_duration[job], end, assigments[(e, job)],  f'E{e}_start_{chr(job)}')
      all_tasks[job, e] = Task(start=start, end=end, interval=interval)
      
  # One task at once for each elf:
  for e in range(nb_elfs):
    model.AddNoOverlap([all_tasks[(j,e)].interval for j in jobs])
    
  # Precedences constraints:
  for (first, second) in order_constraints:
    for e1 in range(nb_elfs):
      for e2 in range(nb_elfs):
        model.Add(all_tasks[second, e2].start >= all_tasks[first, e1].end).OnlyEnforceIf(assigments[e1, first])
          
  # Makespan objective (= minimize total duration).
  makespan = model.NewIntVar(0, horizon, 'makespan')
  for j, e in all_tasks:
    model.Add(all_tasks[j, e].end <= makespan).OnlyEnforceIf(assigments[e, j])
    
  #model.AddDecisionStrategy([all_tasks[job, e].start for e in range(nb_elfs) for job in sorted(jobs) ], cp_model.CHOOSE_LOWEST_MIN, cp_model.SELECT_MIN_VALUE)
    
  model.Minimize(makespan)
  
  # Solve model
  solver = cp_model.CpSolver()
  solver.num_search_workers = 2
  #solver.parameters.search_branching = cp_model.FIXED_SEARCH  # Required to enforce decision strategy
  #status = solver.Solve(model)
  solution_printer = cp_model.ObjectiveSolutionPrinter()
  status = solver.SolveWithSolutionCallback(model, solution_printer)
  
  print('Status: ' + solver.StatusName(status))
  print(f"Time: {solver.WallTime() * 1000:.2f} ms")
  print('Makespan: %i' % solver.Value(makespan))
  
  df_assigments = pd.DataFrame({chr(j) : [assigments[e, j] for e in range(nb_elfs)] for j in jobs}).applymap(solver.Value)
  df_starts = pd.DataFrame(all_tasks, index=Task._fields).T['start'].unstack(0).rename(columns=chr).applymap(solver.Value).where(df_assigments!=0)
  
  return solver.Value(makespan), df_assigments, df_starts

In [0]:
total_time, df_assigments, df_starts = day_7_part2(instructions, nb_elfs=2, min_task_duration=0)
#total_time, df_assigments, df_starts = day_7_part2(open('input.txt').read(), nb_elfs=2, min_task_duration=0)
df_starts

Solution 0, time = 0.004150 s, objective = [21, 14]
Solution 1, time = 0.005888 s, objective = [17, 14]
Solution 2, time = 0.006964 s, objective = [15, 14]
Status: OPTIMAL
Time: 6.55 ms
Makespan: 15


,A,B,C,D,E,F
0,3.0,4.0,0.0,6.0,NaN,NaN
1,NaN,NaN,NaN,NaN,10.0,3.0


In [0]:
#@title
large_instructions = """
Step G must be finished before step L can begin.
Step X must be finished before step U can begin.
Step W must be finished before step H can begin.
Step M must be finished before step S can begin.
Step Z must be finished before step N can begin.
Step K must be finished before step U can begin.
Step V must be finished before step B can begin.
Step L must be finished before step P can begin.
Step U must be finished before step S can begin.
Step D must be finished before step Q can begin.
Step C must be finished before step Q can begin.
Step O must be finished before step N can begin.
Step E must be finished before step P can begin.
Step J must be finished before step Q can begin.
Step R must be finished before step A can begin.
Step P must be finished before step Q can begin.
Step H must be finished before step F can begin.
Step I must be finished before step Y can begin.
Step F must be finished before step T can begin.
Step T must be finished before step Q can begin.
Step S must be finished before step B can begin.
Step A must be finished before step N can begin.
Step B must be finished before step N can begin.
Step Q must be finished before step Y can begin.
Step N must be finished before step Y can begin.
Step G must be finished before step S can begin.
Step S must be finished before step Q can begin.
Step A must be finished before step Y can begin.
Step Q must be finished before step N can begin.
Step Z must be finished before step K can begin.
Step F must be finished before step A can begin.
Step F must be finished before step Q can begin.
Step M must be finished before step V can begin.
Step B must be finished before step Y can begin.
Step A must be finished before step Q can begin.
Step F must be finished before step B can begin.
Step S must be finished before step N can begin.
Step G must be finished before step B can begin.
Step C must be finished before step T can begin.
Step Z must be finished before step D can begin.
Step P must be finished before step N can begin.
Step Z must be finished before step P can begin.
Step K must be finished before step O can begin.
Step R must be finished before step P can begin.
Step J must be finished before step R can begin.
Step W must be finished before step B can begin.
Step T must be finished before step S can begin.
Step M must be finished before step B can begin.
Step K must be finished before step B can begin.
Step I must be finished before step S can begin.
Step H must be finished before step A can begin.
Step O must be finished before step J can begin.
Step H must be finished before step I can begin.
Step I must be finished before step N can begin.
Step D must be finished before step J can begin.
Step P must be finished before step B can begin.
Step T must be finished before step N can begin.
Step D must be finished before step A can begin.
Step M must be finished before step D can begin.
Step R must be finished before step I can begin.
Step U must be finished before step Y can begin.
Step P must be finished before step S can begin.
Step R must be finished before step B can begin.
Step G must be finished before step C can begin.
Step U must be finished before step C can begin.
Step O must be finished before step F can begin.
Step Z must be finished before step E can begin.
Step B must be finished before step Q can begin.
Step E must be finished before step J can begin.
Step X must be finished before step B can begin.
Step O must be finished before step A can begin.
Step H must be finished before step Y can begin.
Step T must be finished before step Y can begin.
Step U must be finished before step H can begin.
Step A must be finished before step B can begin.
Step D must be finished before step Y can begin.
Step X must be finished before step D can begin.
Step V must be finished before step U can begin.
Step L must be finished before step J can begin.
Step G must be finished before step X can begin.
Step Z must be finished before step J can begin.
Step L must be finished before step R can begin.
Step U must be finished before step F can begin.
Step O must be finished before step S can begin.
Step F must be finished before step S can begin.
Step C must be finished before step F can begin.
Step L must be finished before step I can begin.
Step C must be finished before step I can begin.
Step P must be finished before step Y can begin.
Step R must be finished before step H can begin.
Step P must be finished before step I can begin.
Step J must be finished before step B can begin.
Step D must be finished before step S can begin.
Step C must be finished before step E can begin.
Step W must be finished before step J can begin.
Step D must be finished before step T can begin.
Step G must be finished before step D can begin.
Step Z must be finished before step A can begin.
Step U must be finished before step R can begin.
Step P must be finished before step T can begin.
Step C must be finished before step Y can begin.
"""

### Large input - 2 Elfs - min_task_duration = 0

In [0]:
total_time, df_assigments, df_starts = day_7_part2(large_instructions, nb_elfs=2, min_task_duration=0)
df_starts

### Large input - 2 Elfs - min_task_duration = 60

In [0]:
total_time, df_assigments, df_starts = day_7_part2(large_instructions, nb_elfs=2, min_task_duration=60)
df_starts

Status: OPTIMAL
Time: 4267.32 ms
Makespan: 1223


,A,B,C,D,E,F,G,H,I,J,...,Q,R,S,T,U,V,W,X,Y,Z
0,NaN,925.0,356.0,211.0,419.0,700.0,0.0,632.0,NaN,NaN,...,987.0,554.0,846.0,766.0,275.0,NaN,NaN,NaN,NaN,NaN
1,843.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,773.0,484.0,...,NaN,NaN,NaN,NaN,NaN,170.0,252.0,86.0,1138.0,0.0


### Large input - 5 Elfs - min_task_duration = 60

In [0]:
total_time, df_assigments, df_starts = day_7_part2(large_instructions, nb_elfs=5, min_task_duration=60) # should return 1105
df_starts